# Attention GAN

In [3]:
# imports

# for file handling
import os, glob, shutil

# to compress directories into zipfiles
import zipfile

# display file link
from IPython.display import FileLink

# cv2 for image processing
import cv2

# import matplotlib for plotting
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
def compress_folder(source_dir_path, target_dir_path, zip_filename = None):
    '''
    This function compresses the folder into a zipfile and stores it at a specific location.

    Parameters:
    - source_dir_path: path of the directory to zip
    - target_dir_path: path to the directory to save the zip file in
    - zip_filename: name of the zip file

    Returns:
    - path to zip file path
    '''
    # Create a filename for the zip file based on the name of the source directory or the zip_filename
    source_dir_name = os.path.basename(source_dir_path)
    if zip_filename:
        zip_filename = zip_filename + '.zip'
    
    # Create a full path for the zip file in the target directory
    zip_path = os.path.join(target_dir_path, zip_filename)
    
    print("Zipping " + source_dir_path)
    
    # Create a ZipFile object and add the contents of the source directory to it
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        for root, dirs, files in os.walk(source_dir_path):
            for file in files:
                file_path = os.path.join(root, file)
                zip_file.write(file_path, os.path.relpath(file_path, source_dir_path))
    
    print("Saved .zip file to: " + zip_path)
    
    # Return the relative path to the zip file
    return zip_path

# Preparing the Repo

In [5]:
! git clone https://github.com/Ha0Tang/AttentionGAN

Cloning into 'AttentionGAN'...
remote: Enumerating objects: 1150, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 1150 (delta 19), reused 28 (delta 15), pack-reused 1115
Receiving objects: 100% (1150/1150), 211.76 MiB | 25.91 MiB/s, done.
Resolving deltas: 100% (515/515), done.


In [6]:
# move to the repo dir
! cd AttentionGAN && pwd

/kaggle/working/AttentionGAN


# Installing the dependencies

In [9]:
# create a virtual env
! cd AttentionGAN && pip install -r requirements.txt

In [10]:
# create the results directory to store the generated results
results_dir = os.path.join(os.getcwd(), 'results')
! mkdir $results_dir

mkdir: cannot create directory ‘/kaggle/working/results’: File exists


# Results on pre-trained Models

# `horse2zebra`

In [9]:
# download the horse2zebra dataset using the script
! cd AttentionGAN && sh ./datasets/download_cyclegan_dataset.sh horse2zebra

./datasets/download_cyclegan_dataset.sh: 3: [[: not found
Specified [horse2zebra]
for details.

--2023-04-21 20:28:19--  https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/horse2zebra.zip
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.244.190
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.244.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 116867962 (111M) [application/zip]
Saving to: ‘./datasets/horse2zebra.zip’

./datasets/horse2ze 100%[===================>] 111.45M  59.6MB/s    in 1.9s    

2023-04-21 20:28:21 (59.6 MB/s) - ‘./datasets/horse2zebra.zip’ saved [116867962/116867962]

Archive:  ./datasets/horse2zebra.zip
   creating: ./datasets/horse2zebra/trainA/
  inflating: ./datasets/horse2zebra/trainA/n02381460_6223.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_1567.jpg  
  inflating: ./datasets/horse2zebra/trainA/n02381460_3354.jpg  
  inflating: ./datasets/horse2zebra/train

In [11]:
# download the pretrained model trained on horse2zebra dataset
! cd AttentionGAN && sh ./scripts/download_attentiongan_model.sh horse2zebra

Note: available models are apple2orange, horse2zebra, photo2map, cezanne2photo, monet2photo, selfie2anime
Specified [horse2zebra]
for details.

--2023-04-21 20:28:45--  http://disi.unitn.it/~hao.tang/uploads/models/AttentionGAN/horse2zebra_pretrained.tar.gz
Resolving disi.unitn.it (disi.unitn.it)... 193.205.194.4
Connecting to disi.unitn.it (disi.unitn.it)|193.205.194.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87764623 (84M) [application/x-gzip]
Saving to: ‘./checkpoints/horse2zebra_pretrained.tar.gz’

./checkpoints/horse 100%[===================>]  83.70M  19.0MB/s    in 5.5s    

2023-04-21 20:28:51 (15.3 MB/s) - ‘./checkpoints/horse2zebra_pretrained.tar.gz’ saved [87764623/87764623]

horse2zebra_pretrained/
horse2zebra_pretrained/latest_net_G_A.pth
horse2zebra_pretrained/latest_net_G_B.pth


In [12]:
# NOTE: the pretrained model is saved at `AttentionGAN/results/horse2zebra_pretrained/test_latest/images` directory

# generate the images
! cd AttentionGAN && python test.py --dataroot ./datasets/horse2zebra --name horse2zebra_pretrained --model attention_gan --dataset_mode unaligned --norm instance --phase test --no_dropout --load_size 256 --crop_size 256 --batch_size 1 --gpu_ids 0 --num_test 5000 --epoch latest --saveDisk

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./datasets/horse2zebra        	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                            

In [ ]:
# create directory to store generated images
results_horse2zebra = os.path.join(results_dir, 'horse2zebra')
! mkdir $results_horse2zebra

In [14]:
# set the save directory containing the original, generated and attention masks
horse2zebra_results = os.path.join(os.getcwd(), 'AttentionGAN', 'results', 'horse2zebra_pretrained', 'test_latest/images')
horse2zebra_results

'/kaggle/working/AttentionGAN/results/horse2zebra_pretrained/test_latest/images'

In [15]:
# list the images in the save directory
! cd AttentionGAN/results/horse2zebra_pretrained/test_latest/images && ls

n02381460_1000_a10_a.png   n02381460_2870_a10_a.png   n02381460_5090_a10_a.png
n02381460_1000_a10_b.png   n02381460_2870_a10_b.png   n02381460_5090_a10_b.png
n02381460_1000_fake_A.png  n02381460_2870_fake_A.png  n02381460_5090_fake_A.png
n02381460_1000_fake_B.png  n02381460_2870_fake_B.png  n02381460_5090_fake_B.png
n02381460_1000_real_A.png  n02381460_2870_real_A.png  n02381460_5090_real_A.png
n02381460_1000_real_B.png  n02381460_2870_real_B.png  n02381460_5090_real_B.png
n02381460_1010_a10_a.png   n02381460_2890_a10_a.png   n02381460_50_a10_a.png
n02381460_1010_a10_b.png   n02381460_2890_a10_b.png   n02381460_50_a10_b.png
n02381460_1010_fake_A.png  n02381460_2890_fake_A.png  n02381460_50_fake_A.png
n02381460_1010_fake_B.png  n02381460_2890_fake_B.png  n02381460_50_fake_B.png
n02381460_1010_real_A.png  n02381460_2890_real_A.png  n02381460_50_real_A.png
n02381460_1010_real_B.png  n02381460_2890_real_B.png  n02381460_50_real_B.png
n02381460_1030_a10_a.png   n02381460_2940_a10_a.png   n0

In [19]:
horse2zebra_results_zip_path = compress_folder(horse2zebra_results, results_horse2zebra, 'horse2zebra_results')

Zipping /kaggle/working/AttentionGAN/results/horse2zebra_pretrained/test_latest/images
Saved .zip file to: /kaggle/working/results/horse2zebra/horse2zebra_results.zip


In [21]:
! cd /kaggle/working/results/horse2zebra && ls

horse2zebra_results.zip


In [22]:
# create zipfile download link
FileLink(r'results/horse2zebra/horse2zebra_results.zip')

/kaggle/working/results/horse2zebra/horse2zebra_results.zip

# `apple2orange`

In [25]:
# download the apple2orange dataset using the script
! cd AttentionGAN && sh ./datasets/download_cyclegan_dataset.sh apple2orange

./datasets/download_cyclegan_dataset.sh: 3: [[: not found
Specified [apple2orange]
for details.

--2023-04-21 21:07:11--  https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/apple2orange.zip
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.244.190
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.244.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78456409 (75M) [application/zip]
Saving to: ‘./datasets/apple2orange.zip’

./datasets/apple2or 100%[===================>]  74.82M  52.6MB/s    in 1.4s    

2023-04-21 21:07:13 (52.6 MB/s) - ‘./datasets/apple2orange.zip’ saved [78456409/78456409]

Archive:  ./datasets/apple2orange.zip
   creating: ./datasets/apple2orange/trainA/
  inflating: ./datasets/apple2orange/trainA/n07740461_6908.jpg  
  inflating: ./datasets/apple2orange/trainA/n07740461_7635.jpg  
  inflating: ./datasets/apple2orange/trainA/n07740461_586.jpg  
  inflating: ./datasets/apple2orange/

In [26]:
# download the pretrained model trained on apple2orange dataset
! cd AttentionGAN && sh ./scripts/download_attentiongan_model.sh apple2orange

Note: available models are apple2orange, horse2zebra, photo2map, cezanne2photo, monet2photo, selfie2anime
Specified [apple2orange]
for details.

--2023-04-21 21:09:51--  http://disi.unitn.it/~hao.tang/uploads/models/AttentionGAN/apple2orange_pretrained.tar.gz
Resolving disi.unitn.it (disi.unitn.it)... 193.205.194.4
Connecting to disi.unitn.it (disi.unitn.it)|193.205.194.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87890096 (84M) [application/x-gzip]
Saving to: ‘./checkpoints/apple2orange_pretrained.tar.gz’

./checkpoints/apple 100%[===================>]  83.82M  18.8MB/s    in 5.5s    

2023-04-21 21:09:57 (15.2 MB/s) - ‘./checkpoints/apple2orange_pretrained.tar.gz’ saved [87890096/87890096]

apple2orange_pretrained/
apple2orange_pretrained/latest_net_G_A.pth
apple2orange_pretrained/latest_net_G_B.pth


In [27]:
# NOTE: the pretrained model is saved at `AttentionGAN/results/apple2orange_pretrained/test_latest/images` directory

# generate the images
! cd AttentionGAN && python test.py --dataroot ./datasets/apple2orange --name apple2orange_pretrained --model attention_gan --dataset_mode unaligned --norm instance --phase test --no_dropout --load_size 256 --crop_size 256 --batch_size 1 --gpu_ids 0 --num_test 5000 --epoch latest --saveDisk

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./datasets/apple2orange       	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                            

In [28]:
# create directory to store generated images
results_apple2orange = os.path.join(results_dir, 'apple2orange')
! mkdir $results_apple2orange

In [29]:
# set the save directory containing the original, generated and attention masks
apple2orange_results = os.path.join(os.getcwd(), 'AttentionGAN', 'results', 'apple2orange_pretrained', 'test_latest/images')
apple2orange_results

'/kaggle/working/AttentionGAN/results/apple2orange_pretrained/test_latest/images'

In [30]:
# list the images in the save directory
! cd AttentionGAN/results/apple2orange_pretrained/test_latest/images && ls

n07740461_10011_a10_a.png   n07740461_2770_a10_a.png
n07740461_10011_a10_b.png   n07740461_2770_a10_b.png
n07740461_10011_fake_A.png  n07740461_2770_fake_A.png
n07740461_10011_fake_B.png  n07740461_2770_fake_B.png
n07740461_10011_real_A.png  n07740461_2770_real_A.png
n07740461_10011_real_B.png  n07740461_2770_real_B.png
n07740461_10080_a10_a.png   n07740461_2840_a10_a.png
n07740461_10080_a10_b.png   n07740461_2840_a10_b.png
n07740461_10080_fake_A.png  n07740461_2840_fake_A.png
n07740461_10080_fake_B.png  n07740461_2840_fake_B.png
n07740461_10080_real_A.png  n07740461_2840_real_A.png
n07740461_10080_real_B.png  n07740461_2840_real_B.png
n07740461_10211_a10_a.png   n07740461_2921_a10_a.png
n07740461_10211_a10_b.png   n07740461_2921_a10_b.png
n07740461_10211_fake_A.png  n07740461_2921_fake_A.png
n07740461_10211_fake_B.png  n07740461_2921_fake_B.png
n07740461_10211_real_A.png  n07740461_2921_real_A.png
n07740461_10211_real_B.png  n07740461_2921_real_B.png
n07740461_1030_a10_a.png    n07740

In [31]:
apple2orange_results_zip_path = compress_folder(apple2orange_results, results_apple2orange, 'apple2orange_results')

Zipping /kaggle/working/AttentionGAN/results/apple2orange_pretrained/test_latest/images
Saved .zip file to: /kaggle/working/results/apple2orange/apple2orange_results.zip


In [32]:
! cd /kaggle/working/results/apple2orange && ls

apple2orange_results.zip


In [34]:
# create zipfile download link
FileLink(r'results/apple2orange/apple2orange_results.zip')

/kaggle/working/results/apple2orange/apple2orange_results.zip

# `photo2map`

In [41]:
# download the photo2map dataset using the script
! cd AttentionGAN && sh ./datasets/download_cyclegan_dataset.sh maps

./datasets/download_cyclegan_dataset.sh: 3: [[: not found
Specified [maps]
for details.

--2023-04-21 21:36:42--  https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/maps.zip
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.244.190
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.244.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1476754427 (1.4G) [application/zip]
Saving to: ‘./datasets/maps.zip’

./datasets/maps.zip 100%[===================>]   1.38G  73.2MB/s    in 20s     

2023-04-21 21:37:03 (69.1 MB/s) - ‘./datasets/maps.zip’ saved [1476754427/1476754427]

Archive:  ./datasets/maps.zip
   creating: ./datasets/maps/trainA/
  inflating: ./datasets/maps/trainA/850_A.jpg  
  inflating: ./datasets/maps/trainA/1054_A.jpg  
  inflating: ./datasets/maps/trainA/847_A.jpg  
  inflating: ./datasets/maps/trainA/1053_A.jpg  
  inflating: ./datasets/maps/trainA/893_A.jpg  
  inflating: ./datasets/maps/t

In [42]:
# download the pretrained model trained on photo2map dataset
! cd AttentionGAN && sh ./scripts/download_attentiongan_model.sh photo2map

Note: available models are apple2orange, horse2zebra, photo2map, cezanne2photo, monet2photo, selfie2anime
Specified [photo2map]
for details.

--2023-04-21 21:37:36--  http://disi.unitn.it/~hao.tang/uploads/models/AttentionGAN/photo2map_pretrained.tar.gz
Resolving disi.unitn.it (disi.unitn.it)... 193.205.194.4
Connecting to disi.unitn.it (disi.unitn.it)|193.205.194.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87760618 (84M) [application/x-gzip]
Saving to: ‘./checkpoints/photo2map_pretrained.tar.gz’

./checkpoints/photo 100%[===================>]  83.69M  18.9MB/s    in 5.4s    

2023-04-21 21:37:42 (15.4 MB/s) - ‘./checkpoints/photo2map_pretrained.tar.gz’ saved [87760618/87760618]

photo2map_pretrained/
photo2map_pretrained/latest_net_G_A.pth
photo2map_pretrained/latest_net_G_B.pth


In [43]:
# NOTE: the pretrained model is saved at `AttentionGAN/results/photo2map_pretrained/test_latest/images` directory

# generate the images
! cd AttentionGAN && python test.py --dataroot ./datasets/maps --name photo2map_pretrained --model attention_gan --dataset_mode unaligned --norm instance --phase test --no_dropout --load_size 256 --crop_size 256 --batch_size 1 --gpu_ids 0 --num_test 5000 --epoch latest --saveDisk

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./datasets/maps               	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                            

In [44]:
# create directory to store generated images
results_photo2map = os.path.join(results_dir, 'photo2map')
! mkdir $results_photo2map

In [45]:
# set the save directory containing the original, generated and attention masks
photo2map_results = os.path.join(os.getcwd(), 'AttentionGAN', 'results', 'photo2map_pretrained', 'test_latest/images')
photo2map_results

'/kaggle/working/AttentionGAN/results/photo2map_pretrained/test_latest/images'

In [46]:
# list the images in the save directory
! cd AttentionGAN/results/photo2map_pretrained/test_latest/images && ls

1000_A_a10_a.png   259_A_fake_B.png  506_A_a10_a.png   753_A_fake_B.png
1000_A_a10_b.png   259_A_real_A.png  506_A_a10_b.png   753_A_real_A.png
1000_A_fake_A.png  259_A_real_B.png  506_A_fake_A.png  753_A_real_B.png
1000_A_fake_B.png  25_A_a10_a.png    506_A_fake_B.png  754_A_a10_a.png
1000_A_real_A.png  25_A_a10_b.png    506_A_real_A.png  754_A_a10_b.png
1000_A_real_B.png  25_A_fake_A.png   506_A_real_B.png  754_A_fake_A.png
1001_A_a10_a.png   25_A_fake_B.png   507_A_a10_a.png   754_A_fake_B.png
1001_A_a10_b.png   25_A_real_A.png   507_A_a10_b.png   754_A_real_A.png
1001_A_fake_A.png  25_A_real_B.png   507_A_fake_A.png  754_A_real_B.png
1001_A_fake_B.png  260_A_a10_a.png   507_A_fake_B.png  755_A_a10_a.png
1001_A_real_A.png  260_A_a10_b.png   507_A_real_A.png  755_A_a10_b.png
1001_A_real_B.png  260_A_fake_A.png  507_A_real_B.png  755_A_fake_A.png
1002_A_a10_a.png   260_A_fake_B.png  508_A_a10_a.png   755_A_fake_B.png
1002_A_a10_b.png   260_A_real_A.png  508_A_a10_b.png   755_A_real_A.

In [47]:
photo2map_results_zip_path = compress_folder(photo2map_results, results_photo2map, 'photo2map_results')

Zipping /kaggle/working/AttentionGAN/results/photo2map_pretrained/test_latest/images
Saved .zip file to: /kaggle/working/results/photo2map/photo2map_results.zip


In [48]:
! cd /kaggle/working/results/photo2map && ls

photo2map_results.zip


In [49]:
# create zipfile download link
FileLink(r'results/photo2map/photo2map_results.zip')

/kaggle/working/results/photo2map/photo2map_results.zip

# `cezanne2photo`

In [52]:
# download the cezzanne2photo dataset using the script
! cd AttentionGAN && sh ./datasets/download_cyclegan_dataset.sh cezanne2photo

./datasets/download_cyclegan_dataset.sh: 3: [[: not found
Specified [cezanne2photo]
for details.

--2023-04-21 21:51:36--  https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/cezanne2photo.zip
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.244.190
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.244.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 279885071 (267M) [application/zip]
Saving to: ‘./datasets/cezanne2photo.zip’

./datasets/cezanne2 100%[===================>] 266.92M  80.7MB/s    in 3.5s    

2023-04-21 21:51:40 (77.1 MB/s) - ‘./datasets/cezanne2photo.zip’ saved [279885071/279885071]

Archive:  ./datasets/cezanne2photo.zip
   creating: ./datasets/cezanne2photo/trainA/
  inflating: ./datasets/cezanne2photo/trainA/00311.jpg  
  inflating: ./datasets/cezanne2photo/trainA/00454.jpg  
  inflating: ./datasets/cezanne2photo/trainA/00461.jpg  
  inflating: ./datasets/cezanne2photo/trainA/00502

In [53]:
# download the pretrained model trained on cezanne2photo dataset
! cd AttentionGAN && sh ./scripts/download_attentiongan_model.sh cezanne2photo

Note: available models are apple2orange, horse2zebra, photo2map, cezanne2photo, monet2photo, selfie2anime
Specified [cezanne2photo]
for details.

--2023-04-21 21:52:53--  http://disi.unitn.it/~hao.tang/uploads/models/AttentionGAN/cezanne2photo_pretrained.tar.gz
Resolving disi.unitn.it (disi.unitn.it)... 193.205.194.4
Connecting to disi.unitn.it (disi.unitn.it)|193.205.194.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87750817 (84M) [application/x-gzip]
Saving to: ‘./checkpoints/cezanne2photo_pretrained.tar.gz’

./checkpoints/cezan 100%[===================>]  83.69M  19.0MB/s    in 5.5s    

2023-04-21 21:52:59 (15.3 MB/s) - ‘./checkpoints/cezanne2photo_pretrained.tar.gz’ saved [87750817/87750817]

cezanne2photo_pretrained/
cezanne2photo_pretrained/latest_net_G_A.pth
cezanne2photo_pretrained/latest_net_G_B.pth


In [54]:
# NOTE: the pretrained model is saved at `AttentionGAN/results/cezanne2photo_pretrained/test_latest/images` directory

# generate the images
! cd AttentionGAN && python test.py --dataroot ./datasets/cezanne2photo --name cezanne2photo_pretrained --model attention_gan --dataset_mode unaligned --norm instance --phase test --no_dropout --load_size 256 --crop_size 256 --batch_size 1 --gpu_ids 0 --num_test 5000 --epoch latest --saveDisk

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./datasets/cezanne2photo      	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                            

In [55]:
# create directory to store generated images
results_cezanne2photo = os.path.join(results_dir, 'cezanne2photo')
! mkdir $results_cezanne2photo

In [56]:
# set the save directory containing the original, generated and attention masks
cezanne2photo_results = os.path.join(os.getcwd(), 'AttentionGAN', 'results', 'cezanne2photo_pretrained', 'test_latest/images')
cezanne2photo_results

'/kaggle/working/AttentionGAN/results/cezanne2photo_pretrained/test_latest/images'

In [57]:
# list the images in the save directory
! cd AttentionGAN/results/cezanne2photo_pretrained/test_latest/images && ls

00010_a10_a.png   00180_fake_B.png  00330_a10_a.png   00470_fake_B.png
00010_a10_b.png   00180_real_A.png  00330_a10_b.png   00470_real_A.png
00010_fake_A.png  00180_real_B.png  00330_fake_A.png  00470_real_B.png
00010_fake_B.png  00190_a10_a.png   00330_fake_B.png  00480_a10_a.png
00010_real_A.png  00190_a10_b.png   00330_real_A.png  00480_a10_b.png
00010_real_B.png  00190_fake_A.png  00330_real_B.png  00480_fake_A.png
00020_a10_a.png   00190_fake_B.png  00340_a10_a.png   00480_fake_B.png
00020_a10_b.png   00190_real_A.png  00340_a10_b.png   00480_real_A.png
00020_fake_A.png  00190_real_B.png  00340_fake_A.png  00480_real_B.png
00020_fake_B.png  00200_a10_a.png   00340_fake_B.png  00490_a10_a.png
00020_real_A.png  00200_a10_b.png   00340_real_A.png  00490_a10_b.png
00020_real_B.png  00200_fake_A.png  00340_real_B.png  00490_fake_A.png
00060_a10_a.png   00200_fake_B.png  00350_a10_a.png   00490_fake_B.png
00060_a10_b.png   00200_real_A.png  00350_a10_b.png   00490_real_A.png
00060_fake

In [58]:
cezanne2photo_results_zip_path = compress_folder(cezanne2photo_results, results_cezanne2photo, 'cezanne2photo_results')

Zipping /kaggle/working/AttentionGAN/results/cezanne2photo_pretrained/test_latest/images
Saved .zip file to: /kaggle/working/results/cezanne2photo/cezanne2photo_results.zip


In [59]:
! cd /kaggle/working/results/cezanne2photo && ls

cezanne2photo_results.zip


In [60]:
# create zipfile download link
FileLink(r'results/cezanne2photo/cezanne2photo_results.zip')

/kaggle/working/results/cezanne2photo/cezanne2photo_results.zip

# `monet2photo`

In [61]:
# download the monet2photo dataset using the script
! cd AttentionGAN && sh ./datasets/download_cyclegan_dataset.sh monet2photo

./datasets/download_cyclegan_dataset.sh: 3: [[: not found
Specified [monet2photo]
for details.

--2023-04-21 22:07:13--  https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/monet2photo.zip
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.244.190
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.244.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 305231073 (291M) [application/zip]
Saving to: ‘./datasets/monet2photo.zip’

./datasets/monet2ph 100%[===================>] 291.09M  67.9MB/s    in 4.5s    

2023-04-21 22:07:17 (65.3 MB/s) - ‘./datasets/monet2photo.zip’ saved [305231073/305231073]

Archive:  ./datasets/monet2photo.zip
   creating: ./datasets/monet2photo/trainA/
  inflating: ./datasets/monet2photo/trainA/01159.jpg  
  inflating: ./datasets/monet2photo/trainA/01048.jpg  
  inflating: ./datasets/monet2photo/trainA/01144.jpg  
  inflating: ./datasets/monet2photo/trainA/00799.jpg  
  inflating: 

In [62]:
# download the pretrained model trained on monet2photo dataset
! cd AttentionGAN && sh ./scripts/download_attentiongan_model.sh monet2photo

Note: available models are apple2orange, horse2zebra, photo2map, cezanne2photo, monet2photo, selfie2anime
Specified [monet2photo]
for details.

--2023-04-21 22:07:40--  http://disi.unitn.it/~hao.tang/uploads/models/AttentionGAN/monet2photo_pretrained.tar.gz
Resolving disi.unitn.it (disi.unitn.it)... 193.205.194.4
Connecting to disi.unitn.it (disi.unitn.it)|193.205.194.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87712000 (84M) [application/x-gzip]
Saving to: ‘./checkpoints/monet2photo_pretrained.tar.gz’

./checkpoints/monet 100%[===================>]  83.65M  18.9MB/s    in 5.4s    

2023-04-21 22:07:46 (15.4 MB/s) - ‘./checkpoints/monet2photo_pretrained.tar.gz’ saved [87712000/87712000]

monet2photo_pretrained/
monet2photo_pretrained/latest_net_G_A.pth
monet2photo_pretrained/latest_net_G_B.pth


In [63]:
# NOTE: the pretrained model is saved at `AttentionGAN/results/monet2photo_pretrained/test_latest/images` directory

# generate the images
! cd AttentionGAN && python test.py --dataroot ./datasets/monet2photo --name monet2photo_pretrained --model attention_gan --dataset_mode unaligned --norm instance --phase test --no_dropout --load_size 256 --crop_size 256 --batch_size 1 --gpu_ids 0 --num_test 5000 --epoch latest --saveDisk

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./datasets/monet2photo        	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                            

In [64]:
# create directory to store generated images
results_monet2photo = os.path.join(results_dir, 'monet2photo')
! mkdir $results_monet2photo

In [65]:
# set the save directory containing the original, generated and attention masks
monet2photo_results = os.path.join(os.getcwd(), 'AttentionGAN', 'results', 'monet2photo_pretrained', 'test_latest/images')
monet2photo_results

'/kaggle/working/AttentionGAN/results/monet2photo_pretrained/test_latest/images'

In [66]:
# list the images in the save directory
! cd AttentionGAN/results/monet2photo_pretrained/test_latest/images && ls

00010_a10_a.png   00320_fake_A.png  00710_real_A.png  01020_a10_a.png
00010_a10_b.png   00320_fake_B.png  00710_real_B.png  01020_a10_b.png
00010_fake_A.png  00320_real_A.png  00730_a10_a.png   01020_fake_A.png
00010_fake_B.png  00320_real_B.png  00730_a10_b.png   01020_fake_B.png
00010_real_A.png  00330_a10_a.png   00730_fake_A.png  01020_real_A.png
00010_real_B.png  00330_a10_b.png   00730_fake_B.png  01020_real_B.png
00020_a10_a.png   00330_fake_A.png  00730_real_A.png  01030_a10_a.png
00020_a10_b.png   00330_fake_B.png  00730_real_B.png  01030_a10_b.png
00020_fake_A.png  00330_real_A.png  00740_a10_a.png   01030_fake_A.png
00020_fake_B.png  00330_real_B.png  00740_a10_b.png   01030_fake_B.png
00020_real_A.png  00350_a10_a.png   00740_fake_A.png  01030_real_A.png
00020_real_B.png  00350_a10_b.png   00740_fake_B.png  01030_real_B.png
00030_a10_a.png   00350_fake_A.png  00740_real_A.png  01040_a10_a.png
00030_a10_b.png   00350_fake_B.png  00740_real_B.png  01040_a10_b.png
00030_fake_A

In [67]:
monet2photo_results_zip_path = compress_folder(monet2photo_results, results_monet2photo, 'monet2photo_results')

Zipping /kaggle/working/AttentionGAN/results/monet2photo_pretrained/test_latest/images
Saved .zip file to: /kaggle/working/results/monet2photo/monet2photo_results.zip


In [68]:
! cd /kaggle/working/results/monet2photo && ls

monet2photo_results.zip


In [69]:
# create zipfile download link
FileLink(r'results/monet2photo/monet2photo_results.zip')

/kaggle/working/results/monet2photo/monet2photo_results.zip

# `selfie2anime`

In [8]:
# save the notebook and restart the kernel before running this
! cd .. && cp -r input/selfie2anime working/AttentionGAN/datasets

In [24]:
# check if the dataset was copied
! cd AttentionGAN/datasets && ls

download_cyclegan_dataset.sh  selfie2anime


In [25]:
# check the contents of models dir
! cd AttentionGAN/models && ls

__init__.py   __pycache__	       attention_gan_model.py  base_model.pyc
__init__.pyc  attention_gan1_model.py  base_model.py	       networks.py


In [26]:
# remove attention_gan_model.py and networks.py (to be replaces with newer files)
! cd AttentionGAN/models && rm attention_gan_model.py && rm networks.py

In [27]:
# check the contents of models dir (after deleting)
! cd AttentionGAN/models && ls

__init__.py   __pycache__	       base_model.py
__init__.pyc  attention_gan1_model.py  base_model.pyc


In [28]:
# copy the attention_gan_model.py and networks.py from AttentionGAN/AttentionGAN-geo dir to the models dir
! cp AttentionGAN/AttentionGAN-geo/attention_gan_model.py AttentionGAN/models && cp AttentionGAN/AttentionGAN-geo/networks.py AttentionGAN/models

In [29]:
# check the contents of AttentionGAN/models dir (after updating)
! cd AttentionGAN/models && ls

__init__.py   __pycache__	       attention_gan_model.py  base_model.pyc
__init__.pyc  attention_gan1_model.py  base_model.py	       networks.py


In [11]:
# download the pretrained model trained on selfie2anime dataset
! cd AttentionGAN && sh ./scripts/download_attentiongan_model.sh selfie2anime

Note: available models are apple2orange, horse2zebra, photo2map, cezanne2photo, monet2photo, selfie2anime
Specified [selfie2anime]
for details.

--2023-04-22 01:48:43--  http://disi.unitn.it/~hao.tang/uploads/models/AttentionGAN/selfie2anime_pretrained.tar.gz
Resolving disi.unitn.it (disi.unitn.it)... 193.205.194.4
Connecting to disi.unitn.it (disi.unitn.it)|193.205.194.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87848644 (84M) [application/x-gzip]
Saving to: ‘./checkpoints/selfie2anime_pretrained.tar.gz’

./checkpoints/selfi 100%[===================>]  83.78M  18.2MB/s    in 5.7s    

2023-04-22 01:48:49 (14.7 MB/s) - ‘./checkpoints/selfie2anime_pretrained.tar.gz’ saved [87848644/87848644]

selfie2anime_pretrained/
selfie2anime_pretrained/latest_net_G_A.pth
selfie2anime_pretrained/latest_net_G_B.pth


In [30]:
# NOTE: the pretrained model is saved at `AttentionGAN/results/selfie2anime_pretrained/test_latest/images` directory

# generate the images
! cd AttentionGAN && python test.py --dataroot ./datasets/selfie2anime --name selfie2anime_pretrained --model attention_gan --dataset_mode unaligned --norm instance --phase test --no_dropout --load_size 256 --crop_size 256 --batch_size 1 --gpu_ids 0 --num_test 5000 --epoch latest --saveDisk

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./datasets/selfie2anime       	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                            

In [31]:
# create directory to store generated images
results_selfie2anime = os.path.join(results_dir, 'selfie2anime')
! mkdir $results_selfie2anime

In [32]:
# set the save directory containing the original, generated and attention masks
selfie2anime_results = os.path.join(os.getcwd(), 'AttentionGAN', 'results', 'selfie2anime_pretrained', 'test_latest/images')
selfie2anime_results

'/kaggle/working/AttentionGAN/results/selfie2anime_pretrained/test_latest/images'

In [33]:
# list the images in the save directory
! cd AttentionGAN/results/selfie2anime_pretrained/test_latest/images && ls

female_10328_a10_a.png	 female_14402_fake_A.png  female_24302_real_A.png
female_10328_a10_b.png	 female_14402_fake_B.png  female_24302_real_B.png
female_10328_fake_A.png  female_14402_real_A.png  female_24446_a10_a.png
female_10328_fake_B.png  female_14402_real_B.png  female_24446_a10_b.png
female_10328_real_A.png  female_14491_a10_a.png   female_24446_fake_A.png
female_10328_real_B.png  female_14491_a10_b.png   female_24446_fake_B.png
female_10500_a10_a.png	 female_14491_fake_A.png  female_24446_real_A.png
female_10500_a10_b.png	 female_14491_fake_B.png  female_24446_real_B.png
female_10500_fake_A.png  female_14491_real_A.png  female_24702_a10_a.png
female_10500_fake_B.png  female_14491_real_B.png  female_24702_a10_b.png
female_10500_real_A.png  female_14587_a10_a.png   female_24702_fake_A.png
female_10500_real_B.png  female_14587_a10_b.png   female_24702_fake_B.png
female_11025_a10_a.png	 female_14587_fake_A.png  female_24702_real_A.png
female_11025_a10_b.png	 female_14587_fake_B.png

In [34]:
selfie2anime_results_zip_path = compress_folder(selfie2anime_results, results_selfie2anime, 'selfie2anime_results')

Zipping /kaggle/working/AttentionGAN/results/selfie2anime_pretrained/test_latest/images
Saved .zip file to: /kaggle/working/results/selfie2anime/selfie2anime_results.zip


In [35]:
! cd /kaggle/working/results/selfie2anime && ls

selfie2anime_results.zip


In [36]:
# create zipfile download link
FileLink(r'results/selfie2anime/selfie2anime_results.zip')

/kaggle/working/results/selfie2anime/selfie2anime_results.zip

# Results on Trained Models

# `vangogh2photo`

In [8]:
# download the vangogh2photo dataset using the script
# ! cd AttentionGAN && sh ./datasets/download_cyclegan_dataset.sh vangogh2photo

./datasets/download_cyclegan_dataset.sh: 3: [[: not found
Specified [vangogh2photo]
for details.

--2023-04-22 05:14:34--  https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/vangogh2photo.zip
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.244.190
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.244.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 306590349 (292M) [application/zip]
Saving to: ‘./datasets/vangogh2photo.zip’

./datasets/vangogh2 100%[===================>] 292.39M  94.2MB/s    in 3.2s    

2023-04-22 05:14:38 (91.1 MB/s) - ‘./datasets/vangogh2photo.zip’ saved [306590349/306590349]

Archive:  ./datasets/vangogh2photo.zip
   creating: ./datasets/vangogh2photo/trainA/
  inflating: ./datasets/vangogh2photo/trainA/00311.jpg  
  inflating: ./datasets/vangogh2photo/trainA/00454.jpg  
  inflating: ./datasets/vangogh2photo/trainA/00461.jpg  
  inflating: ./datasets/vangogh2photo/trainA/00176

In [9]:
# train AttentionGAN on vangogh2photo dataset
# ! cd AttentionGAN && python train.py --dataroot ./datasets/vangogh2photo/ --name vangogh2photo_attentiongan --model attention_gan --dataset_mode unaligned --pool_size 50 --no_dropout --norm instance --lambda_A 10 --lambda_B 10 --lambda_identity 0.5 --load_size 286 --crop_size 256 --batch_size 4 --niter 100 --niter_decay 100 --gpu_ids 0 --display_id 0 --display_freq 100 --print_freq 100

----------------- Options ---------------
               batch_size: 4                             	[default: 1]
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: ./datasets/vangogh2photo/     	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 100                           	[default: 400]
               display_id: 0                             	[default: 1]
            display_ncols: 10                            
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: late

# `grumpifycat`

In [11]:
# download the grumpifycat dataset using the script
! cd AttentionGAN && sh ./datasets/download_cyclegan_dataset.sh grumpifycat

./datasets/download_cyclegan_dataset.sh: 3: [[: not found
Specified [grumpifycat]
for details.

--2023-04-22 06:35:29--  https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/grumpifycat.zip
Resolving people.eecs.berkeley.edu (people.eecs.berkeley.edu)... 128.32.244.190
Connecting to people.eecs.berkeley.edu (people.eecs.berkeley.edu)|128.32.244.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20399194 (19M) [application/zip]
Saving to: ‘./datasets/grumpifycat.zip’

./datasets/grumpify 100%[===================>]  19.45M  20.5MB/s    in 0.9s    

2023-04-22 06:35:31 (20.5 MB/s) - ‘./datasets/grumpifycat.zip’ saved [20399194/20399194]

Archive:  ./datasets/grumpifycat.zip
   creating: ./datasets/grumpifycat/trainA/
  inflating: ./datasets/grumpifycat/trainA/Russian_Blue_102_cat0.jpg  
  inflating: ./datasets/grumpifycat/trainA/Russian_Blue_111_cat0.jpg  
  inflating: ./datasets/grumpifycat/trainA/Russian_Blue_113_cat0.jpg  
  inflating: ./datasets/

In [12]:
# train AttentionGAN on grumpifycat dataset
! cd AttentionGAN && python train.py --dataroot ./datasets/grumpifycat/ --name grumpifycat_attentiongan --model attention_gan --dataset_mode unaligned --pool_size 50 --no_dropout --norm instance --lambda_A 10 --lambda_B 10 --lambda_identity 0.5 --load_size 286 --crop_size 256 --batch_size 4 --niter 100 --niter_decay 100 --gpu_ids 0 --display_id 0 --display_freq 500 --print_freq 500

----------------- Options ---------------
               batch_size: 4                             	[default: 1]
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: ./datasets/grumpifycat/       	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 500                           	[default: 400]
               display_id: 0                             	[default: 1]
            display_ncols: 10                            
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: late

In [14]:
! cd AttentionGAN/checkpoints/grumpifycat_attentiongan && ls

100_net_D_A.pth  145_net_G_A.pth  195_net_D_A.pth  5_net_G_A.pth
100_net_D_B.pth  145_net_G_B.pth  195_net_D_B.pth  5_net_G_B.pth
100_net_G_A.pth  150_net_D_A.pth  195_net_G_A.pth  60_net_D_A.pth
100_net_G_B.pth  150_net_D_B.pth  195_net_G_B.pth  60_net_D_B.pth
105_net_D_A.pth  150_net_G_A.pth  200_net_D_A.pth  60_net_G_A.pth
105_net_D_B.pth  150_net_G_B.pth  200_net_D_B.pth  60_net_G_B.pth
105_net_G_A.pth  155_net_D_A.pth  200_net_G_A.pth  65_net_D_A.pth
105_net_G_B.pth  155_net_D_B.pth  200_net_G_B.pth  65_net_D_B.pth
10_net_D_A.pth	 155_net_G_A.pth  20_net_D_A.pth   65_net_G_A.pth
10_net_D_B.pth	 155_net_G_B.pth  20_net_D_B.pth   65_net_G_B.pth
10_net_G_A.pth	 15_net_D_A.pth   20_net_G_A.pth   70_net_D_A.pth
10_net_G_B.pth	 15_net_D_B.pth   20_net_G_B.pth   70_net_D_B.pth
110_net_D_A.pth  15_net_G_A.pth   25_net_D_A.pth   70_net_G_A.pth
110_net_D_B.pth  15_net_G_B.pth   25_net_D_B.pth   70_net_G_B.pth
110_net_G_A.pth  160_net_D_A.pth  25_net_G_A.pth   75_net_D_A.pth
110_net_G_B.pth 

In [16]:
# testing the trained model to generate images
! cd AttentionGAN && python test.py --dataroot ./datasets/grumpifycat/ --name grumpifycat_attentiongan --model attention_gan --dataset_mode unaligned --norm instance --phase test --no_dropout --load_size 256 --crop_size 256 --batch_size 1 --gpu_ids 0 --num_test 5000 --epoch latest

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./checkpoints                 
                crop_size: 256                           
                 dataroot: ./datasets/grumpifycat/       	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
                load_iter: 0                            

In [17]:
! cd AttentionGAN/datasets/grumpifycat/ && ls

trainA	trainB
